# Courier

In [1]:
import pandas as pd

## Database connection

In [2]:
import yaml
from sqlalchemy import create_engine


with open("../config.yml", "r") as file:
	config = yaml.safe_load(file)
	config_OLTP = config["OLTP"]
	config_OLAP = config["OLAP"]


url_OLTP = (f"{config_OLTP['drivername']}://{config_OLTP['user']}:{config_OLTP['password']}"
			f"@{config_OLTP['host']}:{config_OLTP['port']}/{config_OLTP['database_name']}")

url_OLAP = (f"{config_OLAP['drivername']}://{config_OLAP['user']}:{config_OLAP['password']}"
			f"@{config_OLAP['host']}:{config_OLAP['port']}/{config_OLAP['database_name']}")

OLTP_connection = create_engine(url_OLTP)
OLAP_connection = create_engine(url_OLAP)

## Extraction

In [3]:
courier_dimension = pd.read_sql_table("clientes_mensajeroaquitoy", OLTP_connection)
city = pd.read_sql_table("ciudad", OLTP_connection)

In [4]:
courier_dimension.head(1)

,id,user_id,activo,fecha_entrada,fecha_salida,salario,telefono,ciudad_operacion_id,token_Firebase,url_foto
0,1,2,True,NaT,NaT,NaN,310-300000,13.0,None,http:


In [5]:
city.head(1)

,ciudad_id,nombre,departamento_id
0,6,BUGA,1


## Transformation

In [6]:
courier_dimension = courier_dimension.merge(
		city, left_on="ciudad_operacion_id", right_on="ciudad_id", how="left"
)
courier_dimension.head(1)

,id,user_id,activo,fecha_entrada,fecha_salida,salario,telefono,ciudad_operacion_id,token_Firebase,url_foto,ciudad_id,nombre,departamento_id
0,1,2,True,NaT,NaT,NaN,310-300000,13.0,None,http:,13.0,ACOPI YUMBO,1.0


In [7]:
courier_dimension.rename(
	columns={
		"nombre": "courier_city"
	}, inplace=True
)

In [8]:
courier_dimension = courier_dimension[["courier_city"]]

In [9]:
courier_dimension.reset_index(inplace=True)
courier_dimension.rename(columns={ "index": "courier_id" }, inplace=True)
courier_dimension.set_index("courier_id", inplace=True)

In [10]:
courier_dimension.head(1)

,courier_city
courier_id,
0,ACOPI YUMBO


## Load

In [12]:
courier_dimension.to_sql(
	"COURIER_DIMENSION", OLAP_connection, if_exists="replace", index=True
)

INFO:sqlalchemy.engine.Engine:select pg_catalog.version()
INFO:sqlalchemy.engine.Engine:[raw sql] {}
INFO:sqlalchemy.engine.Engine:select current_schema()
INFO:sqlalchemy.engine.Engine:[raw sql] {}
INFO:sqlalchemy.engine.Engine:show standard_conforming_strings
INFO:sqlalchemy.engine.Engine:[raw sql] {}
INFO:sqlalchemy.engine.Engine:BEGIN (implicit)
INFO:sqlalchemy.engine.Engine:SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
INFO:sqlalchemy.engine.Engine:[generated in 0.00099s] {'table_name': 'COURIER_DIMENSION', 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_

50